# TP15 : Algorithme des $k$-moyennes, diminution du nombre de couleurs
On veut réduire à $k$ le nombre de couleurs différentes dans une image, dans le but de la compresser. Pour cela, il faut choisir les $k$ couleurs à considérer, c'est-à-dire trouver $k$ classes d'équivalence parmi les couleurs présentes dans une image. Nous allons utiliser l'algorithme des $k$-moyennes à cette fin.

L'encodage de couleurs RGB ne permet pas de définir une distance simple entre deux couleurs, de sorte que deux couleurs "proches" en terme de distance se ressemblent à l'œil. Nous allons utiliser donc l'encodage HSV, aussi connu sous le nom [Teinte, Saturation, Valeur](https://fr.wikipedia.org/wiki/Teinte_Saturation_Valeur).

Dans tout l'énoncé, une couleur sera un triplet `(h, s, v)` et une image sera un tableau bidimensionnel de couleurs.

Grossièrement, la _teinte_ correspond à la couleur (à un angle sur le disque des couleurs) et la _valeur_ permet de faire varier entre une couleur foncée et une couleur claire. Nous allons décréter que la distance entre deux couleurs $(h_1, s_1, v_1)$ et $(h_2, s_2, v_2)$ est

$$\sqrt{(h_1-h_2)^2+0.2\times (v_1-v_2)^2}\:.$$

On va classifier les pixels sur $k$ classes en fonction de leurs couleurs, en se servant de cette distance.

### Exercice 1 : calcul de distance
Écrire une fonction `distance_couleurs` qui prend en argument deux couleurs au format HSV en renvoie leur distance selon la formule précédente.

In [ ]:
import math
epsilon = 0.000001
assert(math.fabs(distance_couleurs((45,912,-24.23), (84,-781,748)) - 347.5468811254102) < epsilon)

### Exercice 2 : couleur la plus proche
Dans l'algorithme des $k$-moyennes, on dispose d'un ensemble de points qui permettent de partitionner l'ensemble des données: chaque donnée est rattachée au plus proche parmi ces points.
Écrire une fonction `plus_proche` qui prend en argument une liste de couleurs au format HSV et une couleur `c` à ce même format, et renvoie la couleur la plus proche de `c` dans la liste, pour la distance de l'exercice précédent.

In [ ]:
L = [(155, 113, 159),
     (174, 36, 234),
     (133, 155, 140),
     (183, 251, 77),
     (177, 231, 252),
     (145, 22, 120),
     (230, 244, 137),
     (68, 173, 155),
     (184, 165, 168),
     (58, 110, 164),
     (241, 250, 132),
     (1, 6, 105),
     (205, 125, 53)]
assert(plus_proche(L, (230, 244, 137)) == (230, 244, 137))
assert(plus_proche(L, (230, 0, 137)) == (230, 244, 137))
assert(plus_proche(L, (177, 231, 240)) == (174, 36, 234))
assert(plus_proche(L, (145, 113, 150)) == (155, 113, 159))
assert(plus_proche(L, (199, 0, 53)) == (205, 125, 53))

### Exercice 3 : initialisation des classes
Dans l'algorithme des $k$-moyennes, on commence par prendre $k$ points au hasard. Dans ce TP, les points considérés sont des couleurs. Écrire une fonction `echantillon_couleurs` qui prend en argument une image (donc un tableau bidimensionnel de couleurs au format HSV) et un entier `k` et renvoie une liste de `k` couleurs de l'image selon l'algorithme suivant:
* on tire une liste de `k` pixels distincts au hasard dans l'image (pensez à `random.sample()`),
* on récupère les couleurs de ces pixels.

( _Stricto sensu_ , on ne tire pas $k$ couleurs au hasard, mais on va faire comme si, cela n'a pas grande importance pour l'algorithme des $k$-moyennes de toutes façons.)

In [ ]:
import random
img = [ [ (random.randint(0, 15), random.randint(0, 15), random.randint(0, 15)) for _ in range(15) ] for _ in range(25) ]
echantillon = echantillon_couleurs(img, 17)
assert(len(echantillon) == 17)
for pixel in echantillon:
    assert(any(pixel in ligne for ligne in img))

### Exercice 4 : barycentre
Dans l'algorithme des $k$-moyennes, une fois qu'on a partitionné les données en fonction de leurs distances respectives aux $k$ points, on met à jour l'ensemble de $k$ points, en prenant le barycentre de chaque part. Pour rappel, dans le problème présent, un point est en fait une couleur, donc il faut calculer le barycentre des couleurs des pixels de chaque part.

Écrire une fonction `barycentre` qui prend en argument une part (c'est-à-dire une liste de pixels) et une image (c'est-à-dire un tableau bidimensionnel de couleurs) et renvoie la couleur barycentre des couleurs de ces pixels.

In [ ]:
import math
epsilon = 0.000001
L = [(8, 4), (3, 2), (9, 1), (4, 2)]
img = [[(5, 15, 13), (14, 9, 15), (2, 14, 13), (11, 1, 15), (14, 7, 3), (2, 4, 13)],
 [(10, 8, 8), (2, 12, 4), (5, 10, 0), (14, 14, 6), (7, 12, 13), (8, 2, 6)],
 [(15, 10, 0), (4, 3, 12), (9, 7, 6), (11, 13, 5), (11, 7, 15), (10, 7, 3)],
 [(14, 10, 5), (9, 0, 15), (15, 10, 12), (15, 12, 13), (13, 3, 11), (11, 12, 8)],
 [(6, 4, 10), (15, 8, 1), (2, 7, 7), (12, 0, 14), (1, 9, 6), (9, 6, 10)],
 [(9, 5, 13), (15, 14, 12), (0, 9, 3), (12, 0, 1), (2, 2, 5), (7, 3, 6)],
 [(2, 12, 10), (11, 15, 9), (4, 12, 15), (7, 7, 13), (14, 7, 2), (14, 2, 12)],
 [(10, 4, 6), (2, 5, 15), (4, 2, 6), (1, 11, 10), (0, 9, 13), (14, 15, 14)],
 [(5, 14, 3), (11, 9, 3), (2, 6, 10), (3, 13, 7), (15, 6, 2), (9, 5, 0)],
 [(0, 0, 15), (3, 15, 11), (12, 9, 8), (10, 2, 11), (5, 2, 13), (9, 5, 14)]]

x, y, z = barycentre(L, img)
assert(math.fabs(x - 8.75) < epsilon and math.fabs(y - 9.5) < epsilon and math.fabs(z - 8) < epsilon)

### Exercice 5 : partitionnement
L'étape principale dans l'algorithme des $k$-moyennes est celle du partitionnement: on a un ensemble de points et un ensemble de données, et on souhaite partitionner les données en fonction de leur distance à ces points.

Écrire une fonction `repartition_population` qui prend en argument une liste `couleurs` de couleurs qui sont les points de référence et une image, et renvoie un dictionnaire dont les clefs sont les couleurs de la liste `couleurs` et la valeur associée à une clef `c` est la liste de pixels de l'image dont `c` est la couleur la plus proche dans `couleurs` (un pixel = un couple de coordonnées).

### Exercice 6 : $k$-moyennes
Il s'agit maintenant d'implémenter l'algorithme des $k$-moyennes. Pour rappel:
* on choisit un échantillon de $k$ points au hasard dans les données;
* on partitionne les données en $k$ parts en fonction de leurs distances aux points de l'échantillon;
* on met à jour l'échantillon avec les barycentres des parts;
* on recommence jusqu'à obtenir un échantillon stable.

Pour tester que deux listes contiennent les mêmes éléments, il suffit de convertir les listes en ensembles (`set(liste)`) et de faire un test d'égalité.

Écrire une fonction `k_moyennes` qui prend en argument une image et un entier `k` et renvoie une partition des points de l'image en suivant l'algorithme des $k$-moyennes appliqués aux couleurs des pixels.

### Exercice 7 : recoloration
Écrire une fonction `recolorie` qui prend en argument une image (sous forme de tableau bidimensionnel HSV) et une partition des pixels (sous la forme d'un dictionnaire, comme renvoyé par la fonction `k_moyennes`) et renvoie une image aux mêmes dimensions, mais où la couleur d'un pixel est la clef de la liste à laquelle le pixel appartient dans le dictionnaire. Attention à la complexité, choisissez bien vos boucles!

### Interlude : lecture et affichage des images en HSV
Pour voir le résultat de votre travail, je vous fournis les fonctions suivantes:
* `img2arrayHSV` qui prend un nom de fichier contenant une image en argument et renvoie un tableau bidimensionnel avec les couleurs au format HSV;
* `affiche_imageHSV` qui prend en argument une image sous la forme d'un tableau bidimensionnel de pixels au format HSV et affiche l'image correspondante.

(Je passe par du RGB parce que je n'ai pas réussi à passer directement par du HSV. Toutes mes tentatives se sont soldées par des couleurs bizarres.)

In [ ]:
from PIL import Image
import colorsys

def arrayRGB2arrayHSV(img):
    for i in range(len(img)):
        for j in range(len(img[i])):
            img[i][j] = colorsys.rgb_to_hsv(img[i][j][0]/255, img[i][j][1]/255, img[i][j][2]/255)
            
def img2arrayHSV(fic):
    """fic est le nom d'un fichier contenant une image
    la fonction renvoie une liste de listes de triplets au format HSV"""
    im = Image.open(fic).convert('RGB')  ## chargement
    tab = [ [0 for _ in range(im.height)] for _ in range(im.width)]
    i = 0
    j = -1
    for c in im.getdata():
        if i % im.width == 0: ## debut d'une nouvelle ligne dans l'image
            i = 0
            j += 1
        tab[i][j] = c
        i += 1
    im.close()
    arrayRGB2arrayHSV(tab)
    return tab


            
def arrayHSV2arrayRGB(img):
    img2 = [ [None]*len(img[0]) for _ in range(len(img)) ]
    for i in range(len(img)):
        for j in range(len(img[i])):
            r, g, b = colorsys.hsv_to_rgb(img[i][j][0], img[i][j][1], img[i][j][2])
            img2[i][j] = (int(255*r), int(255*g), int(255*b))
    return img2
            
def affiche_imageHSV(img):
    """img : image sous forme d'une liste de listes de triplets au format HSV
    affichage de l'image (image modifiée)"""
    img = arrayHSV2arrayRGB(img)
    im = Image.new(size=(len(img), len(img[0])), mode='RGB')
    for i in range(len(img)):
        for j in range(len(img[i])):
            im.putpixel((i, j), img[i][j])
    display(im)

### Test
On peut (enfin) tester...

L'image d'origine:

In [ ]:
img = img2arrayHSV('the_ponte_salario_1952.5.50.jpg')
affiche_imageHSV(img)

L'image réduite à 64 couleurs (un peu long):

In [ ]:
n = 64
representants_couleurs = echantillon_couleurs(img, n)
classes_points = k_moyennes(img, n)
img64 = recolorie(img, classes_points)
affiche_imageHSV(img64)

L'image réduite à 32 couleurs (un peu long):

In [ ]:
n = 32
representants_couleurs = echantillon_couleurs(img, n)
classes_points = k_moyennes(img, n)

img32 = recolorie(img, classes_points)
affiche_imageHSV(img32)

L'image réduite à 16 couleurs (un peu long):

In [ ]:
n = 16
representants_couleurs = echantillon_couleurs(img, n)
classes_points = k_moyennes(img, n)
img16 = recolorie(img, classes_points)
affiche_imageHSV(img16)

Mais combien y avait-il de couleurs différentes dans l'image d'origine?

In [ ]:
len(set(pixel for ligne in img for pixel in ligne))